# Pandas: Data Manipulation, Merging, and Intro to Recommendation Logic

## Recap of Session 1

In the previous session, we learned:
*   What Pandas Series and DataFrames are.
*   How to create DataFrames (from dictionaries, loading from CSVs).
*   Inspecting DataFrames (`head`, `tail`, `shape`, `info`, `dtypes`, `describe`).
*   Selecting data (`[]`, `loc`, `iloc`).
*   Filtering data using boolean indexing.
*   Sorting data (`sort_values`).
*   Basic descriptive statistics (`describe`, `nunique`, `value_counts`).

Let's start by re-loading our `movies_df` and importing pandas.

In [1]:
import pandas as pd

# URL for the movies.csv file
movies_url = 'https://drive.google.com/uc?export=download&id=1Uztnn449pnDBDn1XGJPF6uzV34jrP1Te'
movies_df = pd.read_csv(movies_url)

movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Handling Missing Data (Briefly)

Real-world datasets often have missing values, typically represented as `NaN` (Not a Number).

### Checking for Missing Values
`isnull()` returns a DataFrame of booleans indicating True for missing values. `sum()` can then count them per column.

In [2]:
movies_df.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

Our `movies_df` is quite clean. If we had missing values, common strategies include:
*   `dropna()`: Remove rows or columns with missing values.
*   `fillna()`: Fill missing values with a specific value (e.g., mean, median, mode, or a constant).

Example (we won't run this on `movies_df` as it has no NaNs to demonstrate with directly):
```python
# df_with_nans.dropna(subset=['column_with_nan'], inplace=True) # Drop rows where 'column_with_nan' is NaN
# df_with_nans['numeric_col_with_nan'].fillna(df_with_nans['numeric_col_with_nan'].mean(), inplace=True) # Fill with mean
```

## Adding and Modifying Columns

### Adding a New Column
You can add a new column by assigning values to it. If you assign a single value, it will be broadcast to all rows.

In [3]:
movies_df_copy = movies_df.copy() # Work on a copy to keep original movies_df clean for later merging
movies_df_copy['source'] = 'MovieLens'
movies_df_copy.head()

,movieId,title,genres,source
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,MovieLens
1,2,Jumanji (1995),Adventure|Children|Fantasy,MovieLens
2,3,Grumpier Old Men (1995),Comedy|Romance,MovieLens
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,MovieLens
4,5,Father of the Bride Part II (1995),Comedy,MovieLens


You can also create a new column based on existing columns.
Let's try to extract the year from the 'title' column. Movie titles are often in the format "Title (Year)".

In [4]:
# Using .str.extract() with a regular expression to get the year
# This regex looks for 4 digits inside parentheses at the end of the string
movies_df_copy['year_extracted'] = movies_df_copy['title'].str.extract(r'\((\d{4})\)$')
movies_df_copy.head()

,movieId,title,genres,source,year_extracted
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,MovieLens,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,MovieLens,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,MovieLens,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,MovieLens,1995
4,5,Father of the Bride Part II (1995),Comedy,MovieLens,1995


r'...' - This is a raw string in Python, which means backslashes are treated literally. This is important for regex patterns to avoid having to escape backslashes.

\( - Matches a literal opening parenthesis. The backslash is needed because parentheses have special meaning in regex (they're used for grouping).

(\d{4}) - This is a capturing group that matches exactly 4 digits:
* \d matches any digit (0-9)
* {4} specifies exactly 4 occurrences of the preceding pattern
* The parentheses () create a capturing group, which means this part will be extracted

\) - Matches a literal closing parenthesis. Again, the backslash is needed to escape the special meaning of parentheses in regex.

$ - Matches the end of the string

Notice `year_extracted` is a string. We might want to convert it to a number.
Also, some movies might not have a year in the title in this format, leading to `NaN`.

In [5]:
print("Data type of year_extracted:", movies_df_copy['year_extracted'].dtype)
print("Number of NaNs in year_extracted:", movies_df_copy['year_extracted'].isnull().sum())

# Convert to numeric, coercing errors to NaN (for titles that didn't match the pattern)
movies_df_copy['year_extracted'] = pd.to_numeric(movies_df_copy['year_extracted'], errors='coerce')
print("\nData type after conversion:", movies_df_copy['year_extracted'].dtype)
movies_df_copy.head()

Data type of year_extracted: object
Number of NaNs in year_extracted: 24

Data type after conversion: float64


,movieId,title,genres,source,year_extracted
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,MovieLens,1995.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,MovieLens,1995.0
2,3,Grumpier Old Men (1995),Comedy|Romance,MovieLens,1995.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,MovieLens,1995.0
4,5,Father of the Bride Part II (1995),Comedy,MovieLens,1995.0


### Modifying an Existing Column
Let's say we want to clean the movie titles by removing the year part.
We can use `.str.replace()`.

In [6]:
# This regex replaces ' (xxxx)' at the end of the string with an empty string, then trims whitespace
movies_df_copy['title_cleaned'] = movies_df_copy['title'].str.replace(r'\s*\(\d{4}\)$', '', regex=True).str.strip()
movies_df_copy[['title', 'title_cleaned', 'year_extracted']].head()

,title,title_cleaned,year_extracted
0,Toy Story (1995),Toy Story,1995.0
1,Jumanji (1995),Jumanji,1995.0
2,Grumpier Old Men (1995),Grumpier Old Men,1995.0
3,Waiting to Exhale (1995),Waiting to Exhale,1995.0
4,Father of the Bride Part II (1995),Father of the Bride Part II,1995.0


### Dropping Columns
Use the `drop()` method. `axis=1` indicates we're dropping columns ( `axis=0` for rows).

In [7]:
# Let's drop the 'source' column we added
movies_df_temp = movies_df_copy.drop('source', axis=1) # Returns a new DataFrame
# movies_df_copy.drop('source', axis=1, inplace=True) # To modify in place
movies_df_temp.head()

,movieId,title,genres,year_extracted,title_cleaned
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,Toy Story
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995.0,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,1995.0,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995.0,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,1995.0,Father of the Bride Part II


## Merging and Joining DataFrames

Pandas provides ways to combine DataFrames, similar to SQL JOINs. The `pd.merge()` function is most common.

Let's load the `ratings.csv` dataset, which contains user ratings for movies.

In [8]:
ratings_url = 'https://drive.google.com/uc?export=download&id=15IOZXK7f8nvfxhUXap9VEnZDzBTyu2xI'
ratings_df = pd.read_csv(ratings_url)
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


We have `movies_df` (movieId, title, genres) and `ratings_df` (userId, movieId, rating, timestamp).
We can merge them on the common column `movieId`.

In [9]:
merged_df = pd.merge(movies_df, ratings_df, on='movieId')
merged_df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


By default, `pd.merge()` performs an **inner join**: only rows where the key (`movieId`) exists in *both* DataFrames are kept.
Other join types:
*   `how='left'`: Keep all keys from the left DataFrame, fill with NaN if no match in right.
*   `how='right'`: Keep all keys from the right DataFrame.
*   `how='outer'`: Keep all keys from both DataFrames.

Let's check the shape:

In [10]:
print("Shape of movies_df:", movies_df.shape)
print("Shape of ratings_df:", ratings_df.shape)
print("Shape of merged_df (inner join):", merged_df.shape)

# The number of rows in merged_df should be equal to ratings_df if all movieIds in ratings_df are in movies_df
# And if each movieId is unique in movies_df (which it is, as it's a primary key for movies).

Shape of movies_df: (9742, 3)
Shape of ratings_df: (100836, 4)
Shape of merged_df (inner join): (100836, 6)


## Grouping Data: `groupby()`

The `groupby()` method is used for splitting the data into groups based on some criteria, applying a function to each group independently, and then combining the results.

Let's calculate the average rating for each movie.

In [11]:
# Group by 'title' (or 'movieId') and calculate the mean of 'rating' for each group
average_ratings = merged_df.groupby('title')['rating'].mean()
print(type(average_ratings)) # This is a Series
average_ratings.head()

<class 'pandas.core.series.Series'>


title
'71 (2014)                                 4.0
'Hellboy': The Seeds of Creation (2004)    4.0
'Round Midnight (1986)                     3.5
'Salem's Lot (2004)                        5.0
'Til There Was You (1997)                  4.0
Name: rating, dtype: float64

We can convert this Series back to a DataFrame and sort it.

In [12]:
average_ratings_df = average_ratings.reset_index() # Converts 'title' from index to column
average_ratings_df = average_ratings_df.rename(columns={'rating': 'avg_rating'})
top_rated_movies = average_ratings_df.sort_values(by='avg_rating', ascending=False)
top_rated_movies.head()

,title,avg_rating
3354,Gena the Crocodile (1969),5.0
8951,True Stories (1986),5.0
1986,Cosmic Scrat-tastrophe (2015),5.0
5269,Love and Pigeons (1985),5.0
7033,Red Sorghum (Hong gao liang) (1987),5.0


### Grouping by Multiple Columns and Multiple Aggregations
Let's also count the number of ratings per movie, as very popular movies might have slightly lower average ratings than niche movies with few, highly positive ratings.

In [13]:
movie_stats_df = merged_df.groupby('title')['rating'].agg(['mean', 'count'])
movie_stats_df.columns = ['avg_rating', 'num_ratings'] # Rename columns
movie_stats_df = movie_stats_df.reset_index()
movie_stats_df.head()

,title,avg_rating,num_ratings
0,'71 (2014),4.0,1
1,'Hellboy': The Seeds of Creation (2004),4.0,1
2,'Round Midnight (1986),3.5,2
3,'Salem's Lot (2004),5.0,1
4,'Til There Was You (1997),4.0,2


Now sort by number of ratings to see most popular, or by average rating for highly-rated.

In [14]:
print("Most popular movies (by number of ratings):")
print(movie_stats_df.sort_values(by='num_ratings', ascending=False).head())

print("\nTop rated movies (min 50 ratings to avoid niche bias):")
min_ratings_threshold = 50
print(movie_stats_df[movie_stats_df['num_ratings'] >= min_ratings_threshold].sort_values(by='avg_rating', ascending=False).head())

Most popular movies (by number of ratings):
                                 title  avg_rating  num_ratings
3158               Forrest Gump (1994)    4.164134          329
7593  Shawshank Redemption, The (1994)    4.429022          317
6865               Pulp Fiction (1994)    4.197068          307
7680  Silence of the Lambs, The (1991)    4.161290          279
5512                Matrix, The (1999)    4.192446          278

Top rated movies (min 50 ratings to avoid niche bias):
                                                  title  avg_rating  \
7593                   Shawshank Redemption, The (1994)    4.429022   
3499                              Godfather, The (1972)    4.289062   
3011                                  Fight Club (1999)    4.272936   
1961                              Cool Hand Luke (1967)    4.271930   
2531  Dr. Strangelove or: How I Learned to Stop Worr...    4.268041   

      num_ratings  
7593          317  
3499          192  
3011          218  
1961     

## Introduction to Recommendation Logic (Filter-Based)

Now that we have a combined dataset (`movie_stats_df` merged with original `movies_df` to get genres), we can build a very simple recommendation system based on filtering.

First, let's merge `movie_stats_df` (which has `title`, `avg_rating`, `num_ratings`) back with the original `movies_df` (which has `movieId`, `title`, `genres`) to get a comprehensive DataFrame for recommendations.

In [15]:
# We need 'movieId' and 'genres' from movies_df, and 'avg_rating', 'num_ratings' from movie_stats_df.
# Both have 'title', so we can merge on that. Or merge movie_stats_df on movies_df using movieId if stats were grouped by movieId.

# Let's re-calculate movie_stats based on movieId for a cleaner merge
movie_stats_by_id_df = merged_df.groupby('movieId')['rating'].agg(['mean', 'count']).reset_index()
movie_stats_by_id_df.columns = ['movieId', 'avg_rating', 'num_ratings']

recommender_df = pd.merge(movies_df, movie_stats_by_id_df, on='movieId')
recommender_df.head()

,movieId,title,genres,avg_rating,num_ratings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930,215
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818,110
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615,52
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143,7
4,5,Father of the Bride Part II (1995),Comedy,3.071429,49


### Simple Recommender Function
Let's create a function that recommends movies based on:
1.  A chosen genre.
2.  A minimum number of ratings (popularity filter).
3.  A minimum average rating.
And returns the top N movies.

In [16]:
def simple_recommender(df, genre, min_num_ratings=50, min_avg_rating=3.5, top_n=5):
    # Filter by genre
    filtered_df = df[df['genres'].str.contains(genre, case=False, na=False)]
    
    # Filter by minimum number of ratings
    filtered_df = filtered_df[filtered_df['num_ratings'] >= min_num_ratings]
    
    # Filter by minimum average rating
    filtered_df = filtered_df[filtered_df['avg_rating'] >= min_avg_rating]
    
    # Sort by average rating (descending)
    recommended_movies = filtered_df.sort_values(by='avg_rating', ascending=False)
    
    return recommended_movies[['title', 'genres', 'avg_rating', 'num_ratings']].head(top_n)

### Get Recommendations

In [17]:
print("Recommendations for 'Adventure' movies:")
adventure_recs = simple_recommender(recommender_df, genre='Adventure', min_num_ratings=100, min_avg_rating=4.0)
print(adventure_recs)

Recommendations for 'Adventure' movies:
                                                 title  \
898                         Princess Bride, The (1987)   
224          Star Wars: Episode IV - A New Hope (1977)   
897  Star Wars: Episode V - The Empire Strikes Back...   
899  Raiders of the Lost Ark (Indiana Jones and the...   
862             Monty Python and the Holy Grail (1975)   

                                      genres  avg_rating  num_ratings  
898  Action|Adventure|Comedy|Fantasy|Romance    4.232394          142  
224                  Action|Adventure|Sci-Fi    4.231076          251  
897                  Action|Adventure|Sci-Fi    4.215640          211  
899                         Action|Adventure    4.207500          200  
862                 Adventure|Comedy|Fantasy    4.161765          136  


In [18]:
print("\nRecommendations for 'Comedy' movies:")
comedy_recs = simple_recommender(recommender_df, genre='Comedy', min_num_ratings=150, min_avg_rating=3.8, top_n=7)
print(comedy_recs)


Recommendations for 'Comedy' movies:
                          title  \
257         Pulp Fiction (1994)   
314         Forrest Gump (1994)   
520                Fargo (1996)   
968   Back to the Future (1985)   
0              Toy Story (1995)   
3189               Shrek (2001)   

                                                 genres  avg_rating  \
257                         Comedy|Crime|Drama|Thriller    4.197068   
314                            Comedy|Drama|Romance|War    4.164134   
520                         Comedy|Crime|Drama|Thriller    4.116022   
968                             Adventure|Comedy|Sci-Fi    4.038012   
0           Adventure|Animation|Children|Comedy|Fantasy    3.920930   
3189  Adventure|Animation|Children|Comedy|Fantasy|Ro...    3.867647   

      num_ratings  
257           307  
314           329  
520           181  
968           171  
0             215  
3189          170  


In [19]:
print("\nRecommendations for 'Sci-Fi' movies:")
scifi_recs = simple_recommender(recommender_df, genre='Sci-Fi', min_num_ratings=80, top_n=3)
print(scifi_recs)


Recommendations for 'Sci-Fi' movies:
                                                  title  \
224           Star Wars: Episode IV - A New Hope (1977)   
897   Star Wars: Episode V - The Empire Strikes Back...   
1938                                 Matrix, The (1999)   

                       genres  avg_rating  num_ratings  
224   Action|Adventure|Sci-Fi    4.231076          251  
897   Action|Adventure|Sci-Fi    4.215640          211  
1938   Action|Sci-Fi|Thriller    4.192446          278  


## Conclusion & Further Learning

Pandas is a vast library, and we've only scratched the surface. Key takeaways:
*   DataFrames are central for structured data.
*   Inspection, selection, and filtering are fundamental operations.
*   Merging allows combining related datasets.
*   `groupby()` is powerful for aggregations.
*   Even simple filtering can form the basis of a recommendation system.

**Further topics to explore in Pandas:**
*   More advanced indexing (MultiIndex).
*   Time series data handling.
*   `apply()` for more complex row/column-wise operations.
*   `pivot_table` and `crosstab`.
*   More sophisticated recommendation techniques (e.g., content-based filtering using cosine similarity on genre vectors, collaborative filtering).